# Convolutional Neural Network

In [1]:
import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from util_pytorch import mlp
%matplotlib inline  
%config InlineBackend.figure_format='retina'
print ("PyTorch version:[%s]."%(torch.__version__))

# Device Configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("This notebook use [%s]."%(device))

PyTorch version:[1.6.0].
This notebook use [cuda:0].


### Dataset

In [2]:
train_dataset = datasets.MNIST('../data/mnist_data/',
                             download=True,
                             train=True) 
x_train = train_dataset.data.unsqueeze(1)/255.
y_train = train_dataset.targets
n_train = len(x_train)

test_dataset = datasets.MNIST("../data/mnist_data/", 
                             download=True,
                             train=False)

x_test = test_dataset.data.unsqueeze(1)/255.
y_test = test_dataset.targets
n_test = len(x_test)
print ("n_train:[%d], n_test:[%d], x_dim:[%s], y_dim:[%s]"%
       (n_train,n_test,x_train.shape[1:],y_train.shape))

n_train:[60000], n_test:[10000], x_dim:[torch.Size([1, 28, 28])], y_dim:[torch.Size([60000])]


### ConvNet Class

In [3]:
class ConvNetClsClass(object):
    """
    CNN for classification
    """
    def __init__(self,name='CNN',img_dim=[1,28,28], y_dim=10,
                 filter_sizes=[32,32],kernel_sizes=[3,3],h_dims=[128],
                 USE_BN=True, USE_DROPOUT=True, device=None):
        self.name = name
        self.y_dim = y_dim
        self.img_dim = img_dim
        self.downsample_ratio = len(filter_sizes)*2
        
        self.filter_sizes = filter_sizes
        self.kernel_sizes = kernel_sizes
        self.h_dims = h_dims
        
        self.USE_BN = USE_BN
        self.USE_DROPOUT = USE_DROPOUT
        
        self.device = device
        self.build_model()
        self.main_vars = sum([parameter.numel() for parameter in self.net.parameters()])
        print("[%s] instantiated."%(self.name))
        
    def build_model(self):
        """
        Build model
        """
        # Conv layers
        in_features = self.img_dim[0]
        layer_list = []
        for (filter_size,kernel_size) in zip(self.filter_sizes,self.kernel_sizes):
            layer_list.append(nn.Conv2d(in_features, filter_size, kernel_size=3, stride=1, padding=1))
            layer_list.append(nn.MaxPool2d(kernel_size=2, stride=2))
            if self.USE_BN:
                layer_list.append(nn.BatchNorm2d(filter_size))
            layer_list.append(nn.ReLU(inplace=True))
            in_features = filter_size
        conv_block = nn.Sequential(*layer_list)
        
        # Dense layers
        layer_list = []
        layer_list.append(nn.Flatten())
        in_features = (self.img_dim[1]//self.downsample_ratio) * (self.img_dim[2]//self.downsample_ratio) * in_features
        layer_list.append(mlp(in_features,h_dims=self.h_dims+[self.y_dim],actv=nn.ReLU,out_actv=None,
                                      USE_DROPOUT=True))
        dense = nn.Sequential(*layer_list)
        
        self.net = nn.Sequential(*[conv_block, dense]).to(self.device)
        self.cost = nn.CrossEntropyLoss()
        self.optim = optim.Adam(self.net.parameters(), lr=0.001)
        
        
    def update(self, x_batch, y_batch):
        """
        Update model 
        """
        y_pred = self.net(x_batch)
        cost_val = self.cost(y_pred, y_batch)
        self.optim.zero_grad()
        cost_val.backward()
        self.optim.step()
        return cost_val
    
    def get_accr(self, x, y, batch_size):
        """
        Test the model
        """
        n_test = len(x_test)
        p_idx = np.random.permutation(n_test)
        max_iter = np.ceil(n_test/batch_size).astype(np.int) # number of iterations
        with torch.no_grad():
            test_loss = 0
            total = 0
            correct = 0
            for it in range(max_iter):
                b_idx = p_idx[batch_size*(it):batch_size*(it+1)]
                x_batch, y_batch = x[b_idx].to(self.device), y[b_idx].to(self.device)
                y_pred = self.net(x_batch)
                _, predicted = torch.max(y_pred.data, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()
            acc = (100 * correct / total)
        return acc
        
print ("Ready.")

Ready.


### Instantiate Model

In [4]:
C = ConvNetClsClass(name='CNN',y_dim=10,img_dim=[1,28,28],
                    filter_sizes=[32,32],kernel_sizes=[3,3],h_dims=[128],
                    USE_BN=True,USE_DROPOUT=True, device=device)

[CNN] instantiated.


In [5]:
for v_idx,(name, var) in enumerate(C.net.named_parameters()):
    print (v_idx, name, var.shape)

0 0.0.weight torch.Size([32, 1, 3, 3])
1 0.0.bias torch.Size([32])
2 0.2.weight torch.Size([32])
3 0.2.bias torch.Size([32])
4 0.4.weight torch.Size([32, 32, 3, 3])
5 0.4.bias torch.Size([32])
6 0.6.weight torch.Size([32])
7 0.6.bias torch.Size([32])
8 1.1.moedl.0.weight torch.Size([128, 1568])
9 1.1.moedl.0.bias torch.Size([128])
10 1.1.moedl.3.weight torch.Size([10, 128])
11 1.1.moedl.3.bias torch.Size([10])


### Loop

In [7]:
max_epoch,batch_size,print_every = 20,128,1
max_iter = np.ceil(n_train/batch_size).astype(np.int) # number of iterations
for epoch in range(max_epoch):
    p_idx = np.random.permutation(n_train)
    cost_val_sum,cnt = 0,0
    for it in range(max_iter):
        b_idx = p_idx[batch_size*(it):batch_size*(it+1)]
        x_batch,y_batch = x_train[b_idx].to(device), y_train[b_idx].to(device)
        cost_val = C.update(x_batch=x_batch,y_batch=y_batch)
        cost_val_sum += cost_val*len(b_idx)
        cnt += len(b_idx)
    cost_val_avg = cost_val_sum / cnt
    if ((epoch%print_every)==0) or (epoch==(max_epoch-1)):
        accr_val = C.get_accr(x_test, y_test, batch_size)
        print ("epoch:[%d/%d] cost:[%.3f] test_accuracy:[%.3f]"%
               (epoch+1,max_epoch,cost_val_avg,accr_val))
print ("Done.")

epoch:[1/20] cost:[0.164] test_accuracy:[97.850]
epoch:[2/20] cost:[0.058] test_accuracy:[98.550]
epoch:[3/20] cost:[0.044] test_accuracy:[98.570]
epoch:[4/20] cost:[0.037] test_accuracy:[98.800]
epoch:[5/20] cost:[0.031] test_accuracy:[98.770]
epoch:[6/20] cost:[0.027] test_accuracy:[98.820]
epoch:[7/20] cost:[0.023] test_accuracy:[98.860]
epoch:[8/20] cost:[0.021] test_accuracy:[98.690]
epoch:[9/20] cost:[0.020] test_accuracy:[98.780]
epoch:[10/20] cost:[0.019] test_accuracy:[98.880]
epoch:[11/20] cost:[0.017] test_accuracy:[98.830]
epoch:[12/20] cost:[0.014] test_accuracy:[98.800]
epoch:[13/20] cost:[0.013] test_accuracy:[98.930]
epoch:[14/20] cost:[0.012] test_accuracy:[98.950]
epoch:[15/20] cost:[0.014] test_accuracy:[98.920]
epoch:[16/20] cost:[0.014] test_accuracy:[98.950]
epoch:[17/20] cost:[0.012] test_accuracy:[98.880]
epoch:[18/20] cost:[0.012] test_accuracy:[98.830]
epoch:[19/20] cost:[0.012] test_accuracy:[98.920]
epoch:[20/20] cost:[0.010] test_accuracy:[98.930]
Done.
